1. Load the packages

In [3]:
import scipy.io
import random
import numpy as np
import torch
import os
import torchvision
import time
from torchvision import models,datasets, transforms
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.datasets import fetch_openml
import torch.nn as nn
from torch.autograd.function import Function
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler


2. Load Office-Caltech-10 dataset


You need to download [Office-Caltech-10 DeCAF-fc6 features](https://drive.google.com/drive/folders/1XAKj5ikc6ygcanexeiWuwvnHMf5rUVGy) generated in [1] to run this code. Then: 


2.1. If you run the code on Colab, you will need to put the data in the corresponding folder of your [Google Drive](https://drive.google.com/drive/u/0/my-drive).


2.2. If you run the code locally, you will need to put the data in the corresponding folder of your device.



[1] Donahue, J., Jia, Y., Vinyals, O., Hoffman, J., Zhang, N., Tzeng, E., & Darrell, T. (2014, January). Decaf: A deep convolutional activation feature for generic visual recognition. In International conference on machine learning (pp. 647-655). PMLR.

In [4]:
# Mount the Google drive, please ignore this cell if you run the code locally.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
amazon = scipy.io.loadmat('/content/drive/MyDrive/office-31/fc6/amazon_fc6.mat')
webcam = scipy.io.loadmat('/content/drive/MyDrive/office-31/fc6/webcam_fc6.mat')
dslr = scipy.io.loadmat('/content/drive/MyDrive/office-31/fc6/dslr_fc6.mat')
cal  = scipy.io.loadmat('/content/drive/MyDrive/office-31/fc6/caltech_decaf.mat')


In [6]:
amazon_fts,amazon_labels=amazon['fts'],amazon['labels'].reshape(-1)-1
webcam_fts,webcam_labels=webcam['fts'],webcam['labels'].reshape(-1)-1
dslr_fts,dslr_labels=dslr['fts'],dslr['labels'].reshape(-1)-1
cal_fts,cal_labels=cal['feas'],cal['labels'].reshape(-1)-1

In [7]:
print(cal_labels)
print(webcam_labels.shape)
print(dslr_labels.shape)
print(cal_labels.shape)
n_class=10

[0 0 0 ... 9 9 9]
(795,)
(498,)
(1123,)


3. Define functions to formulate the training and testing sets

In [19]:
np.random.choice
class_set=np.array([1,2,6,11,12,13,16,17,18,23])-1
#backpack, bike, calculator, headphones, keyboard, laptop-computer, monitor, mouse, mu, and projector

def get_data(domain_name):
  index=np.array([])
  label_for_training=[]
  fts,labels=None, None
  if domain_name=='amazon':
    fts,labels=amazon_fts,amazon_labels
  elif domain_name=='webcam':
    fts,labels=webcam_fts,webcam_labels
  elif domain_name=='dslr':
    fts,labels=dslr_fts,dslr_labels

  j=0
  for i in class_set:
    index=np.hstack((index,np.where(labels==i)[0]))
    for a in np.where(labels==i)[0]:
      label_for_training.append(j)
    j=j+1
  index=index.astype(int)
  return fts[index],np.array(label_for_training)

def get_index(feature,labels,select_num):
  select_list=np.array([])
  remaining_list=np.array([])
  for i in range(10):
    full_list=np.where(labels==i)[0]
    ran_select=np.random.choice(full_list,select_num,replace=False)
    select_list=np.hstack((select_list,ran_select))


    remain_list=np.setdiff1d(full_list,ran_select)
    remaining_list=np.hstack((remaining_list,remain_list))
  return select_list.astype(int),remaining_list.astype(int)

  


def souce_target_split(src_name, tar_name):
  if src_name =='cad':
    src_fea,src_label=np.array(cal_fts),np.array(cal_labels)
  else:
    src_fea,src_label=get_data(src_name)
  if tar_name == 'cad':
    tar_fea,tar_label=np.array(cal_fts),np.array(cal_labels)
  else:
    tar_fea,tar_label=get_data(tar_name)

  if src_name=='amazon':
    scr_number=20
  elif src_name=='cad':
    scr_number=20
  else:
    scr_number=8
  tar_number=3
  
  src_select,src_remain=get_index(src_fea,src_label,scr_number)
  tar_select,tar_remain=get_index(tar_fea,tar_label,tar_number)



  return src_fea[src_select],src_label[src_select],tar_fea[tar_select],tar_label[tar_select],tar_fea[tar_remain],tar_label[tar_remain]



4. Self-defined layer for implementing unweighted center transfer loss (UCTL)

In [20]:
class UCTL(nn.Module):
    def __init__(self, num_classes, feat_dim, size_average=True):
        super(UCTL, self).__init__()
        self.centers = nn.Parameter(torch.randn(num_classes, feat_dim))
        self.centerlossfunc = CenterlossFunc.apply
        self.feat_dim = feat_dim
        self.size_average = size_average

    def forward(self, label, flip_label, feat):
        batch_size = feat.size(0)
        feat = feat.view(batch_size, -1)
        # To check the dim of centers and features
        if feat.size(1) != self.feat_dim:
            raise ValueError("Center's dim: {0} should be equal to input feature's \
                            dim: {1}".format(self.feat_dim,feat.size(1)))
        batch_size_tensor = feat.new_empty(1).fill_(batch_size if self.size_average else 1)
        loss = self.centerlossfunc(feat, label,flip_label, self.centers, batch_size_tensor)
        return loss


class CenterlossFunc(Function):
    @staticmethod
    def forward(ctx, feature, label,flip_label, centers, batch_size):
        ctx.save_for_backward(feature, label,flip_label, centers, batch_size)
        centers_batch = centers.index_select(0, flip_label.long())
        return (feature - centers_batch).pow(2).sum() / (2.0 * batch_size)

    @staticmethod
    def backward(ctx, grad_output):
        feature,label,flip_label,centers,batch_size = ctx.saved_tensors
        flip_centers_batch = centers.index_select(0, flip_label.long())
        flip_diff=flip_centers_batch-feature


        centers_batch = centers.index_select(0, label.long())
        diff = centers_batch - feature
        # init every iteration
        counts = centers.new_ones(centers.size(0))
        ones = centers.new_ones(label.size(0))
        grad_centers = centers.new_zeros(centers.size())
        counts = counts.scatter_add_(0, label.long(), ones)-1+0.0001

        grad_centers.scatter_add_(0, label.unsqueeze(1).expand(feature.size()).long(), diff)
        grad_centers = grad_centers/counts.view(-1, 1)



        return - grad_output * flip_diff / batch_size, None, None,grad_centers / batch_size, None

#Testing function
def main(test_cuda=False):
    print('-'*80)
    device = torch.device("cuda" if test_cuda else "cpu")
    ct = UCTL(20,2,size_average=True).to(device)
    y = torch.Tensor([0,0,2,1,3]).to(device)
    feat = torch.zeros(5,2).to(device).requires_grad_()
    print(list(ct.parameters()))
    
    print(ct.centers.grad)
    out = ct(y,y,feat)
    print(out.item())
    out.backward()


if __name__ == '__main__':
    torch.manual_seed(999)
    if torch.cuda.is_available():
        main(test_cuda=True)

--------------------------------------------------------------------------------
[Parameter containing:
tensor([[-0.2528,  1.4072],
        [ 0.2910,  1.0365],
        [-0.9816, -3.4219],
        [ 1.4910,  0.2422],
        [ 1.4832, -0.3704],
        [ 0.0941,  2.1528],
        [ 0.6271, -1.1666],
        [-0.7862,  0.0759],
        [-0.0086, -0.6568],
        [-1.0011,  0.2992],
        [ 0.6396, -1.0857],
        [-1.6153,  1.5635],
        [ 0.8194,  0.6117],
        [ 0.7602,  1.4788],
        [ 1.9647,  0.9414],
        [ 0.3883, -0.3957],
        [ 0.5920, -2.8563],
        [-0.4750, -0.9978],
        [ 0.0489,  0.9250],
        [-1.2278, -0.9470]], device='cuda:0', requires_grad=True)]
None
2.02020525932312


5. Define the network

In [21]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = nn.Linear(4096, 1024)
        self.act1= nn.PReLU()
        self.layer2 = nn.Linear(1024, 128)
        self.act2= nn.PReLU()
        self.ip2 = nn.Linear(128, 10)
  

    def forward(self, x):
        x = x.view(-1, 4096)
        x = self.layer1(x)
        x = self.act1(x)
        feature = self.layer2(x)
        feature = self.act2(feature)
        ip2 = self.ip2(feature)
        return feature , ip2

    def predict(self, x):
        return self.forward(x)

In [22]:
from torchsummary import summary
RAND_TENSOR = torch.randn(1,4096).cuda()
device = torch.device("cuda")
net = Net().to(device)
output = net(RAND_TENSOR)
print(output)

(tensor([[-0.0677, -0.0192, -0.0805,  0.0260,  0.4609, -0.0321,  0.2013, -0.2339,
         -0.0389, -0.0053, -0.1083, -0.0702, -0.0596, -0.0726, -0.0100, -0.0579,
         -0.0828,  0.2763,  0.1673,  0.2522,  0.0976,  0.2203, -0.0322,  0.2110,
         -0.1069,  0.0598, -0.0717, -0.0466, -0.0096, -0.0692, -0.0309,  0.2684,
          0.1574,  0.1915, -0.0473, -0.0758, -0.0139, -0.0066,  0.1226, -0.0404,
         -0.0078, -0.0276,  0.0065,  0.5099, -0.0108, -0.0239, -0.0147, -0.0595,
          0.0182, -0.0728, -0.0201, -0.0735, -0.0520,  0.1534, -0.0087,  0.2444,
          0.2740,  0.1030,  0.0501,  0.1275, -0.0252,  0.0831,  0.2448, -0.0804,
         -0.0209,  0.1519, -0.0112,  0.1368, -0.0441, -0.0923,  0.3843,  0.2096,
          0.0447,  0.0806,  0.1171, -0.0418,  0.1088, -0.0266, -0.0901, -0.0295,
          0.0962, -0.0610,  0.2959, -0.0715, -0.0467,  0.6762,  0.0219,  0.0324,
         -0.0392, -0.0271, -0.0570, -0.0722, -0.1474,  0.0327, -0.0468,  0.3923,
          0.2851, -0.0778, 

6. Define the function for training and testing 

In [33]:
def trainandtest(model, dataloaders, optimizer,loss_weight):
    n_class=10
    uctl = UCTL(n_class*2, 128).to(device)
    optimzer4center = torch.optim.SGD(uctl.parameters(), lr =0.5)
    n_epoch = 150
    criterion = nn.CrossEntropyLoss()
    early_stop = 20
    since = time.time()
    best_acc = 0
    stop = 0
    max_acc=[]
    for epoch in range(0, n_epoch):
        stop += 1

        for phase in ['src']:
            if phase == 'src':
                model.train()
            else:
                model.eval()
            total_loss, total_center,correct = 0, 0,0
            for inputs, labels,good_ord,flip_ord in dataloaders[phase]:
                inputs, labels,good_ord,flip_ord = inputs.cuda(),labels.cuda(), good_ord.cuda(),flip_ord.cuda()
                optimizer.zero_grad()
                optimzer4center.zero_grad()

                with torch.set_grad_enabled(phase == 'src'):
                    ip1, outputs = model(inputs)
                    softmax=criterion(outputs, labels)
                    centerlss=uctl(good_ord, flip_ord,ip1)
                    loss = softmax + loss_weight * centerlss
                preds = torch.max(outputs, 1)[1]
                if phase == 'src':
                    loss.backward()
                    optimizer.step()
                    optimzer4center.step()
                total_loss += softmax.item() * inputs.size(0)
                total_center +=centerlss.item()
                correct += torch.sum(preds == labels.data)
            epoch_loss = total_loss / len(dataloaders[phase].dataset)
            epoch_acc = correct.double() / len(dataloaders[phase].dataset)
        model.eval()
        correct = 0
        for inputs, labels in dataloaders['tar']:
          inputs, labels= inputs.cuda(),labels.cuda()
          ip1, outputs = model(inputs)
          preds = torch.max(outputs, 1)[1]
          correct += torch.sum(preds == labels.data)
        epoch_acc = correct.double() / len(dataloaders['tar'].dataset)
        if epoch>145:
          print(f'Epoch: [{epoch:02d}/{n_epoch:02d}]---{phase}, softmax_loss: {epoch_loss:.6f}')
          print('Testing acc:',epoch_acc.item())
          print_center = total_center / len(dataloaders[phase])
          print('UCTL:',print_center)
        max_acc.append(epoch_acc)
    print(max(max_acc))
    time_pass = time.time() - since


7. Define evaluation function

In [34]:
def evaluate(domain_src,domain_tar,seeds):
  batch_size = 32
  n_class=10
  random.seed(seeds) #amend if you want

  train_x_scr,train_y_scr,train_x_tar,train_y_tar,test_x_tar,test_y_tar=souce_target_split(domain_src, domain_tar)

  x_train=np.vstack((train_x_scr,train_x_tar))
  y_train=np.hstack((train_y_scr,train_y_tar))


  x_test=test_x_tar
  y_test=test_y_tar
  print(len(x_train))
  print(len(x_test))
  good_order=np.hstack((train_y_scr,train_y_tar+n_class))
  flip_order=np.hstack((train_y_scr+n_class,train_y_tar))
  x_train=torch.from_numpy(x_train)
  y_train=torch.from_numpy(y_train)
  x_test=torch.from_numpy(x_test)
  y_test=torch.from_numpy(y_test)
  good_order=torch.from_numpy(good_order)
  flip_order=torch.from_numpy(flip_order)
  x_train=x_train.float()
  x_test=x_test.float()
  y_train=y_train.long()
  y_test=y_test.long()

  good_order=good_order.long()
  flip_order=flip_order.long()
  # form the dataset
  train_dataset=TensorDataset(x_train,y_train,good_order,flip_order)
  val_dataset=TensorDataset(x_test,y_test)
  train_loader= DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=2)
  test_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True,num_workers=2)
  dataloaders = {'src': train_loader,
               'tar': test_loader}



  model = Net().cuda()
  param_group = []
  learning_rate = 0.0001
  momentum = 5e-4
  for k, v in model.named_parameters():


    if k.__contains__('classifier'):
        param_group += [{'params': v, 'lr': learning_rate}]
    elif k.__contains__('ip'):
        param_group += [{'params': v, 'lr': learning_rate}]

    else:
        param_group += [{'params': v, 'lr': learning_rate}]
  
  optimizer = torch.optim.Adam(param_group)

  trainandtest(model, dataloaders, optimizer,0.01)


8. Example

Example below is the *Amaon and DSLR* domain adaptation task.
You may change the value of variables to get other experimental results.

Note that models will be trained from scratch. Training time should last for around **2 minutes** for one repetition if you use the GPU (GeForce RTX 3090).

Although the results you get may be slightly different from the ones of the manuscript due to randomized initialization, the gap should be small.

In [38]:
#'amazon', 'webcam','dslr','cad'
source_domain='amazon'
target_domain='dslr'
random_seed=1
evaluate(source_domain,target_domain,random_seed)

230
127
Epoch: [146/150]---src, softmax_loss: 0.005252
Testing acc: 0.8818897637795275
UCTL: 4.796296298503876
Epoch: [147/150]---src, softmax_loss: 0.004963
Testing acc: 0.889763779527559
UCTL: 4.461136996746063
Epoch: [148/150]---src, softmax_loss: 0.005597
Testing acc: 0.905511811023622
UCTL: 4.957630217075348
Epoch: [149/150]---src, softmax_loss: 0.005017
Testing acc: 0.8818897637795275
UCTL: 4.388212293386459
tensor(0.9291, device='cuda:0', dtype=torch.float64)


In [40]:
#'amazon', 'webcam','dslr','cad'
source_domain='dslr'
target_domain='amazon'
random_seed=1
evaluate(source_domain,target_domain,random_seed)

110
928
Epoch: [146/150]---src, softmax_loss: 0.007321
Testing acc: 0.8491379310344828
UCTL: 0.585739016532898
Epoch: [147/150]---src, softmax_loss: 0.007259
Testing acc: 0.8491379310344828
UCTL: 0.5982682406902313
Epoch: [148/150]---src, softmax_loss: 0.007121
Testing acc: 0.8491379310344828
UCTL: 0.5811170637607574
Epoch: [149/150]---src, softmax_loss: 0.007021
Testing acc: 0.8469827586206896
UCTL: 0.5707979649305344
tensor(0.8696, device='cuda:0', dtype=torch.float64)
